In [ ]:
# Prob Stmt : To find insights from the data to generate value for the business.
# Client wants to collect and analyse some diff datasets to find some pattern and make music based on that.
# Client wants to collect top trending global songs playlist data on weekly basis. Hence, asked to make ETL pipeline.
# Spotify is a music streaming app.

In [ ]:
                   # Architecture
# Spotify API - (Music Artists,Albums & Tracks) , Get client id & secret key id from dev account to extract data
# Note: With python Application package will connect with Spotify API to extract data
# AWS Lambda - Write and Deploy code on it (Data extraction and data transformation code)
# AWS CloudWatch - Scheduler to run code on weekly basis or specific time or set alarms & monitor logs
# AWS S3 - Store raw data in it 
# Trigger - using it atomatically trigger transformation fn/code to transform data and put back in S3
# Glue Crawler - Once data in S3 it will infer schema to create AWS Glue Data Catalog(metadata repository)
# AWS Athena - Once data in Data Catalog use athena to run SQL queries on top of it or in S3 buckets

In [ ]:
#python library to for Spotify Web API
!pip install spotipy

In [ ]:
pip install --upgrade pip

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
# Provide details for authentication to extract data from spotify
client_credentials_mgr = SpotifyClientCredentials(client_id='',
                                                      client_secret='')

In [ ]:
# Create an object to provides authorization to access data to extract
sp = spotipy.Spotify(client_credentials_manager = client_credentials_mgr)

In [ ]:
playlist_link = 'https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF'

In [ ]:
playlist_URI = playlist_link.split('/')[-1]

In [ ]:
# To extract all the info related to tracks
data = sp.playlist_tracks(playlist_URI)

In [ ]:
data

In [ ]:
# Extracting relevant data one by one
data['items'][0]['track']['album']['id']

In [ ]:
data['items'][0]['track']['album']['name']

In [ ]:
data['items'][0]['track']['album']['release_date']

In [ ]:
data['items'][0]['track']['album']['total_tracks']

In [ ]:
data['items'][0]['track']['album']['external_urls']['spotify']

In [ ]:
data['items']

In [ ]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                    'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)
album_list

In [ ]:
artist_list = []
for row in data['items']:
    for key,value in row.items():
        if key == 'track':
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'],'artist_name':artist['name'],'externam_url':artist['external_urls']['spotify']}
                artist_list.append(artist_dict)
artist_list

In [ ]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_dict = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                'popularity':song_popularity,'song_added':song_added,'album_id':album_id,'artist_id':artist_id}
    song_list.append(song_dict)
song_list

In [ ]:
# Converting above datasets into dfs
album_df = pd.DataFrame.from_dict(album_list)
artist_df = pd.DataFrame.from_dict(artist_list)
song_df = pd.DataFrame.from_dict(song_list)

In [ ]:
album_df = album_df.drop_duplicates(subset='album_id')
artist_df = artist_df.drop_duplicates(subset='artist_id')
song_df = song_df.drop_duplicates(subset='song_id')

In [ ]:
album_df.head()

In [ ]:
artist_df.head()

In [ ]:
song_df.head()

In [ ]:
album_df.info()

In [ ]:
# Applying transformation to convert into datetime datatype

album_df['release_date'] = pd.to_datetime(album_df['release_date'])

In [ ]:
album_df.info()

In [ ]:
song_df['song_added'] = pd.to_datetime(song_df['song_added'])


In [ ]:
song_df.info()